#### **Tratamento de Dados `Visibilímetro`**

> Port of Santos Center of Excellence
- NebulaPredictor


In [3]:
from tqdm import tqdm
import pandas as pd
import PyPDF2
import os

In [12]:
pdf_files = [f for f in os.listdir('raw') if f.endswith('.pdf')]

cpsp_files = [f for f in pdf_files if 'CPSP' in f]
palmas_files = [f for f in pdf_files if 'PALMAS' in f]
brasiltp_files = [f for f in pdf_files if 'BTP' in f]
praticagem_files = [f for f in pdf_files if 'PRATICAGEM' in f]

In [13]:
def process_pdf_files(files):
    all_data = []
    for pdf_file in files:
        with open(os.path.join('raw', pdf_file), 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            content = ""
            for page in tqdm(pdf_reader.pages, desc=f"R :: {pdf_file}"): content += page.extract_text()
        
        rows = content.strip().split('\n')[1:]

        for row in tqdm(rows, desc=f"P :: {pdf_file}"):
            date_time, visibility = row.rsplit(' ', 1)
            date_time = date_time.strip()
            if len(date_time) > 19: date_time=date_time[:19]
            try: visibility_value = float(visibility)
            except ValueError: visibility_value = float('nan')
            all_data.append([date_time, visibility_value])
            
    return all_data

In [14]:
def create_df(data):
    df = pd.DataFrame(data, columns=['DateTime', 'Visibilidade_m'])
    df['DateTime'] = pd.to_datetime(df['DateTime'], format='mixed', dayfirst=True)
    
    df['Ano'] = df['DateTime'].dt.year
    df['Mes'] = df['DateTime'].dt.month 
    df['Dia'] = df['DateTime'].dt.day
    df['Hora'] = df['DateTime'].dt.hour
    df['Minuto'] = df['DateTime'].dt.minute
    df['Segundo'] = df['DateTime'].dt.second
    
    return df[['DateTime', 'Ano', 'Mes', 'Dia', 'Hora', 'Minuto', 'Segundo', 'Visibilidade_m']]

---

In [15]:
def process_pdf_files(files):
    all_data = []
    for pdf_file in files:
        with open(os.path.join('raw', pdf_file), 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            content = ""
            for page in tqdm(pdf_reader.pages, desc=f"Reading {pdf_file}"): 
                content += page.extract_text()
        
        rows = content.strip().split('\n')[1:]

        for row in tqdm(rows, desc=f"Processing {pdf_file}"):
            try: # Validar se a linha tem pelo menos um espaço
                if ' ' not in row: continue
                    
                date_time, visibility = row.rsplit(' ', 1)
                date_time = date_time.strip()
                
                # Validar formato da data (deve ter pelo menos 10 caracteres: DD/MM/YYYY)
                if len(date_time) < 10 or date_time.lower() == 'm': continue
                    
                if len(date_time) > 19: date_time=date_time[:19]
                
                try: visibility_value=float(visibility)
                except ValueError: visibility_value=float('nan')
                    
                all_data.append([date_time, visibility_value])
            
            except Exception: continue
    return all_data

In [16]:
def create_df(data):
    if not data: return pd.DataFrame(columns=['DateTime', 'Ano', 'Mes', 'Dia', 'Hora', 'Minuto', 'Segundo', 'Visibilidade_m'])
        
    df = pd.DataFrame(data, columns=['DateTime', 'Visibilidade_m'])
    df['DateTime'] = pd.to_datetime(df['DateTime'], format='mixed', dayfirst=True, errors='coerce')
    df = df.dropna(subset=['DateTime'])
    
    df['Ano'] = df['DateTime'].dt.year
    df['Mes'] = df['DateTime'].dt.month 
    df['Dia'] = df['DateTime'].dt.day
    df['Hora'] = df['DateTime'].dt.hour
    df['Minuto'] = df['DateTime'].dt.minute
    df['Segundo'] = df['DateTime'].dt.second
    
    return df[['DateTime', 'Ano', 'Mes', 'Dia', 'Hora', 'Minuto', 'Segundo', 'Visibilidade_m']]

In [82]:
df_cpsp = create_df(process_pdf_files(cpsp_files))

Processing VISIBIL_CPSP_102024.pdf: 100%|██████████| 5700/5700 [00:00<00:00, 140389.75it/s]


In [83]:
df_palmas = create_df(process_pdf_files(palmas_files))

Processing VISIBIL_PALMAS_102024.pdf: 100%|██████████| 11299/11299 [00:00<00:00, 118991.16it/s]


In [18]:
df_brasiltp = create_df(process_pdf_files(brasiltp_files))

Processing VISIBIL_BTP_102024.pdf: 100%|██████████| 5700/5700 [00:00<00:00, 405728.18it/s]


In [84]:
df_praticagem = create_df(process_pdf_files(praticagem_files))

Processing VISIBIL_PRATICAGEM_102024.pdf: 100%|██████████| 11699/11699 [00:00<00:00, 244048.69it/s]


In [19]:
df_cpsp.to_csv(r'data\Visibil CPSP 01-10 (2024).csv', index=False, encoding='UTF-8')
df_cpsp.to_excel(r'data\Visibil CPSP 01-10 (2024).xlsx', index=False, engine='openpyxl')

df_palmas.to_csv(r'data\Visibil PALM 06-10 (2024).csv', index=False, encoding='UTF-8')
df_palmas.to_excel(r'data\Visibil PALM 06-10 (2024).xlsx', index=False, engine='openpyxl')

df_praticagem.to_csv(r'data\Visibil PRAT 01-10 (2024).csv', index=False, encoding='UTF-8')
df_praticagem.to_excel(r'data\Visibil PRAT 01-10 (2024).xlsx', index=False, engine='openpyxl')

df_brasiltp.to_csv(r'data\Visibil BRTP 01-10 (2024).csv', index=False, encoding='UTF-8')
df_brasiltp.to_excel(r'data\Visibil BRTP 01-10 (2024).xlsx', index=False, engine='openpyxl')

In [53]:
df_brasiltp.describe()

,DateTime,Ano,Mes,Dia,Hora,Minuto,Segundo,Visibilidade_m
count,58500,58500.0,58500.000000,58500.000000,58500.000000,58500.000000,58500.000000,58500.000000
mean,2024-05-16 23:27:21.959487232,2024.0,5.471795,1.548444,11.152855,29.415453,15.001538,1998.147760
min,2024-01-01 00:00:00,2024.0,1.000000,1.000000,0.000000,0.000000,0.000000,1997.800000
25%,2024-03-01 22:15:22.500000,2024.0,3.000000,1.000000,5.000000,14.000000,0.000000,1998.080000
50%,2024-05-03 01:36:15,2024.0,5.000000,2.000000,11.000000,29.000000,30.000000,1998.140000
75%,2024-08-01 23:30:07.500000,2024.0,8.000000,2.000000,17.000000,44.000000,30.000000,1998.200000
max,2024-10-03 00:22:30,2024.0,10.000000,3.000000,23.000000,59.000000,30.000000,1999.020000
std,NaN,0.0,2.870381,0.559059,7.064304,17.301114,15.000128,0.123013


In [52]:
df_brasiltp.iloc[55500:55560, :]

,DateTime,Ano,Mes,Dia,Hora,Minuto,Segundo,Visibilidade_m
55500,2024-10-01 22:53:30,2024,10,1,22,53,30,1998.14
55501,2024-10-01 22:54:00,2024,10,1,22,54,0,1998.11
55502,2024-10-01 22:54:30,2024,10,1,22,54,30,1998.11
55503,2024-10-01 22:55:00,2024,10,1,22,55,0,1998.05
55504,2024-10-01 22:55:30,2024,10,1,22,55,30,1998.08
55505,2024-10-01 22:56:00,2024,10,1,22,56,0,1998.11
55506,2024-10-01 22:56:30,2024,10,1,22,56,30,1998.08
55507,2024-10-01 22:57:00,2024,10,1,22,57,0,1998.08
55508,2024-10-01 22:57:30,2024,10,1,22,57,30,1998.14
55509,2024-10-01 22:58:00,2024,10,1,22,58,0,1998.11
